In [1]:
import keras

Using TensorFlow backend.


esides Deep Dream, another major development in deep learning-driven image modification that happened in the summer of 2015 is neural style transfer, introduced by Leon Gatys et al. The neural style transfer algorithm has undergone many refinements and spawned many variations since its original introduction, including a viral smartphone app, called Prisma. For simplicity, this section focuses on the formulation described in the original paper.

Neural style transfer consists in applying the "style" of a reference image to a target image, while conserving the "content" of the target image:

'''loss = distance(style(reference_image) - style(generated_image)) +
       distance(content(original_image) - content(generated_image))
'''

Where distance is a norm function such as the L2 norm, content is a function that takes an image and computes a representation of its "content", and style is a function that takes an image and computes a representation of its "style".

Minimizing this loss would cause style(generated_image) to be close to style(reference_image), while content(generated_image) would be close to content(generated_image), thus achieving style transfer as we defined it.

A fundamental observation made by Gatys et al is that deep convolutional neural networks offer precisely a way to mathematically defined the style and content functions. Let's see how.

In [15]:
from keras.preprocessing.image import load_img,img_to_array

target_image_path='E:/photo/avril_cropped.jpg'

style_reference_image='E:/photo/impronte_d_artista_cropped.jpg'

w,h=load_img(target_image_path).size
img_h=400
img_w=int(w*img_h/h)


In [16]:
import numpy as np
from keras.applications import vgg19

In [17]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_h, img_w))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [18]:
from keras import backend as K
target_image=K.constant(preprocess_image(target_image_path))
style_reference_image=K.constant((preprocess_image(style_reference_image)))
combination_image=K.placeholder((1,img_h,img_w,3))
input_tensor =K.concatenate([target_image,style_reference_image,combination_image],axis=0)


In [19]:
model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet',include_top=False)
print('Model loaded')

Model loaded


In [20]:
def content_loss(bass,combination):
    return K.sum(K.square(combination-bass))
def gram_matrix(x):
    features=K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
    gram=K.dot(features,K.transpose(features))
    
    return gram

def style_loss(style,combination):
    S=gram_matrix(style)
    C=gram_matrix(combination)
    size=img_h*img_w
    return K.sum(K.square(S-C))/(4.*(3**2)*(size**2))

def total_variation_loss(x):
    a=K.square(
        x[:,:img_h-1,:img_w-1,:]-x[:,1:,:img_w-1,:])
    
    b=K.square(
    x[:,:img_h-1,:img_w-1,:]-x[:,:img_h-1,1:,:])
    return K.sum(K.pow(a+b,1.25))



In [21]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# Name of layer used for content loss
content_layer = 'block5_conv2'
# Name of layers used for style loss
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# Weights in the weighted average of the loss components
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# Define the loss by adding all components to a `loss` variable
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features,
                                      combination_features)
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)

In [22]:
grads = K.gradients(loss, combination_image)[0]

# Function to fetch the values of the current loss and the current gradients
fetch_loss_and_grads = K.function([combination_image], [loss, grads])


class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_h, img_w, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = 'style_transfer_result'
iterations = 10

# Run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss.
# This is our initial state: the target image.
# Note that `scipy.optimize.fmin_l_bfgs_b` can only process flat vectors.
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x,
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # Save current generated image
    img = x.copy().reshape((img_h, img_w, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 3163619800.0
Image saved as style_transfer_result_at_iteration_0.png
Iteration 0 completed in 89s
Start of iteration 1


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 2279670800.0
Image saved as style_transfer_result_at_iteration_1.png
Iteration 1 completed in 91s
Start of iteration 2
Current loss value: 1876965900.0
Image saved as style_transfer_result_at_iteration_2.png
Iteration 2 completed in 91s
Start of iteration 3
Current loss value: 1655852300.0
Image saved as style_transfer_result_at_iteration_3.png
Iteration 3 completed in 92s
Start of iteration 4
Current loss value: 1518346800.0
Image saved as style_transfer_result_at_iteration_4.png
Iteration 4 completed in 94s
Start of iteration 5
Current loss value: 1418753400.0
Image saved as style_transfer_result_at_iteration_5.png
Iteration 5 completed in 92s
Start of iteration 6
Current loss value: 1342787200.0
Image saved as style_transfer_result_at_iteration_6.png
Iteration 6 completed in 91s
Start of iteration 7
Current loss value: 1286344600.0
Image saved as style_transfer_result_at_iteration_7.png
Iteration 7 completed in 90s
Start of iteration 8
Current loss value: 1242535